In [ ]:
import json
import os
from collections import defaultdict
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt

In [ ]:
class Day:
    def __init__(self, file):
        self.file = file
        self.activities, self.skills = json.loads(self.file.read_text())


def checkAllElementsEqual(lst):
    return len(set(lst)) == 1


class Dataset:
    def __init__(self, days):
        self.days = days

    def getSkillExpLists(
        self, removeDeltaZero=False, minMaxToleranceGT=None, LastXDays=None
    ):
        data = self.days
        if LastXDays is not None:
            days = self.limitDays(LastXDays)
        if removeDeltaZero:
            data = self.removeDeltaZero(days)
        else:
            out = defaultdict(list)
            for item in self.days:
                for skill in self.days[item].skills:
                    out[skill].append(int(self.days[item].skills[skill]["level"]))
            data = dict(out)
        if minMaxToleranceGT is not None:
            data = self.removeMinMaxTollerance(data, minMaxToleranceGT)

        return (days.keys(), data)

    def removeMinMaxTollerance(self, ds, minMaxToleranceGT):
        keep = []
        for skill in ds:
            ma = max(ds[skill])
            mi = min(ds[skill])
            de = int(ma - mi)
            if de > minMaxToleranceGT:
                keep.append(skill)
        new_ds = {}
        for skill in keep:
            new_ds[skill] = ds[skill]
        return dict(new_ds)

    def limitDays(self, LastXDays):
        new_days = {}
        for day in self.days:
            if (datetime.now() - day).days <= LastXDays:
                new_days[day] = self.days[day]
        return new_days

    def getDays(self):
        return list(self.days.keys())

    def removeDeltaZero(self, data):
        values = defaultdict(list)
        for day in data:
            for skill in self.days[day].skills:
                values[skill].append(int(self.days[day].skills[skill]["level"]))
        keep = []
        for skill in values:
            if not checkAllElementsEqual(values[skill]):
                keep.append(skill)
        new_ds = {}
        for skill in keep:
            new_ds[skill] = values[skill]
        return dict(new_ds)


def getDataSet():
    paths = list(
        filter(None, [p if p.is_file() else None for p in Path("data").iterdir()])
    )
    paths = sorted(paths, key=os.path.getmtime)
    return Dataset(
        {
            datetime.strptime(f.name.replace(".json", ""), "%Y-%m-%d"): Day(f)
            for f in list(paths)
        }
    )

# Main

In [ ]:
ds = getDataSet()

In [ ]:
x_Axes, expandedDataset = ds.getSkillExpLists(
    removeDeltaZero=True, minMaxToleranceGT=3, LastXDays=30
)
del expandedDataset["overall"]
x_Axes, expandedDataset

In [ ]:
plt.figure(figsize=(8, 8), facecolor="white")
# Plot a simple line chart
mi = 0
ma = 0
for name in expandedDataset:
    l = expandedDataset[name]
    for item in l:
        mi = min(item, mi)
        ma = max(item, ma)
    plt.plot(x_Axes, l, label=name)
# Plot another line on the same chart/graph
plt.legend(loc="lower left")
plt.xticks(rotation=45)
print(mi, ma)
plt.yticks(list(range(mi, ma, 10)))
plt.grid(True, color="grey", linewidth="1.4", linestyle="-.")

plt.savefig(f'charts/{datetime.now().date().strftime("%m-%d-%Y")}.png')

In [ ]:
datetime.now().date().strftime("%m-%d-%Y")